In [5]:
from eli5 import show_weights, explain_prediction
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

import os
import pickle

PROJ_PATH = r'F:\tmp\data science\UnnParserBot'
# CLASSES   = ['економіка', 'кримінал', 'культура', 'міжнародні новини', 'позитив', 'політика', 'спорт', 'суспільство', 'технології']
CLASSES   = ['економіка', 'кримінал', 'міжнародні новини', 'політика', 'спорт', 'суспільство']

## LogisticRegression

In [2]:
# Loading the dataset, collecting the data
all_ds = load_npz(os.path.join(PROJ_PATH, 'src', 'class_data.npz')).tocsr()

# Loading the target
with open(os.path.join(PROJ_PATH, 'src', 'class_headings.hd'), 'rb') as f:
    headings = pickle.load(f)

# Loading vectorizers to obtain feature names
with open(os.path.join(PROJ_PATH, 'src', 'class_tfidf_w.vct'), 'rb') as f:
    tfidf_w = pickle.load(f)
    
with open(os.path.join(PROJ_PATH, 'src', 'class_tfidf_ch.vct'), 'rb') as f:
    tfidf_ch = pickle.load(f)

# Collecting feature names
tfidf_w_labels = [k for k,v in sorted(list(tfidf_w.vocabulary_.items()), key=lambda x: x[1])]
tfidf_ch_labels = [k for k,v in sorted(list(tfidf_ch.vocabulary_.items()), key=lambda x: x[1])]
orig_features = tfidf_w_labels + tfidf_ch_labels

train_ds, test_ds, y_train, y_test = train_test_split(all_ds,
                                                      headings,
                                                      test_size=0.2,
                                                      stratify=headings,
                                                      random_state=42)

le = LabelEncoder()
le.fit(CLASSES)

LabelEncoder()

In [3]:
def get_prediction_stats(target_test, target_pred=None):
    """
    Prints a map <column_name> -> number of samples.
    If target_pred is provided also gives (<right_predictions>/total) stats.
    """
    
    if target_pred is None:
        print(pd.Series(le.inverse_transform(target_test)).value_counts().sort_index())
    else:
        unique_columns = target_test.unique()
        for column in unique_columns:
            c_pred = len([y for y, yy in zip(target_test, target_pred) if y == column == yy])
            value_counts = target_test.value_counts()[column]
            percentage = (c_pred/value_counts)*100
            print(f'{le.inverse_transform([column])[0]:17s}: {value_counts:5d}, correctly predicted {c_pred:4d} ({percentage:.3f}%)')

In [8]:
# Finding the best parameters using KFold cross validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
classifier = LogisticRegression(solver='saga', C=1)
results_skf = cross_val_score(classifier, all_ds, headings, cv=skf)
print(f'Accuracy: {results_skf.mean()*100.0:.2f}%')

Accuracy: 82.66%


In [12]:
# Training the model
classifier = LogisticRegression(solver='saga', C=1)
log_reg = classifier.fit(train_ds, y_train)

In [13]:
# Predicting
log_predictions = log_reg.predict(test_ds)

In [14]:
# Evaluating with f1 macro score (without class_weight='balanced')
print(f'Test evaluation: {f1_score(y_test, log_predictions, average="weighted")}')
print(f'Train evaluation: {f1_score(y_train, log_reg.predict(train_ds), average="weighted")}')

Test evaluation: 0.8247289726768079
Train evaluation: 0.8952804069375224


In [15]:
# Evaluating with f1 macro score (with class_weight='balanced')
print(f'Test evaluation: {f1_score(y_test, log_predictions, average="weighted")}')
print(f'Train evaluation: {f1_score(y_train, log_reg.predict(train_ds), average="weighted")}')

Test evaluation: 0.8195540320305105
Train evaluation: 0.8794511080550723


In [15]:
print(classification_report(y_test, log_predictions, zero_division=True, target_names=CLASSES))

                   precision    recall  f1-score   support

        економіка       0.78      0.69      0.73       522
         кримінал       0.85      0.88      0.86      1319
міжнародні новини       0.81      0.87      0.84       881
         політика       0.83      0.85      0.84      2076
            спорт       0.99      0.97      0.98       451
      суспільство       0.75      0.66      0.71       951

         accuracy                           0.83      6200
        macro avg       0.83      0.82      0.83      6200
     weighted avg       0.82      0.83      0.82      6200



In [16]:
get_prediction_stats(pd.Series(y_test), target_pred=log_predictions)

суспільство      :   951, correctly predicted  632 (66.456%)
політика         :  2076, correctly predicted 1771 (85.308%)
кримінал         :  1319, correctly predicted 1159 (87.870%)
спорт            :   451, correctly predicted  436 (96.674%)
економіка        :   522, correctly predicted  360 (68.966%)
міжнародні новини:   881, correctly predicted  766 (86.947%)


In [21]:
show_weights(log_reg, 
             target_names=CLASSES,
             feature_names=orig_features,
             top=15)

In [22]:
explain_prediction(log_reg,
                   test_ds.toarray()[6],
                   target_names=CLASSES,
                   feature_names=orig_features,
                   top=15)

Explanation(estimator="LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=100,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=None, solver='saga', tol=0.0001, verbose=0,\n                   warm_start=False)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='економіка', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='посилання даний', weight=0.12406914918800653, std=None, value=0.11261282575687605), FeatureWeight(feature='україна', weight=0.11287939132839518, std=None, value=0.0790719816295066), FeatureWeight(feature='даний', weight=0.0920430821335632, std=None, value=0.10855460211473571), FeatureWeight(feature='повідомляти посилання', weight=0.08229724911982537, std=None, value=0.0899627881231925), FeatureWeight(feature='теплий', weight=0.0693574996492975, std=None, value=0.1343943142581103), FeatureWeight(feature='унн повідомляти', weight=0.05334929720172681, std=None, value=0.08772958857843031), FeatureWeight(feature='стан', weight=0.049020354990827686, std=None, value=0.07710667370878735), FeatureWeight(feature='кінець', weight=0.0470636999847043, std=None, value=0.09947279631444005)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.2431758166732854, std=None, value=1.0), FeatureWeight(feature='30', weight=-0.12757372057827351, std=None, value=0.19150961747498818), FeatureWeight(feature='градус', weight=-0.11563633725052065, std=None, value=0.2251043782160064), FeatureWeight(feature='вночі', weight=-0.09977598863034956, std=None, value=0.2448583920018226), FeatureWeight(feature='північний', weight=-0.055760291368664124, std=None, value=0.18439554835122274), FeatureWeight(feature='решта', weight=-0.047935828365174016, std=None, value=0.22182733515795455), FeatureWeight(feature='південь', weight=-0.047607127324979584, std=None, value=0.13563461287011214)], pos_remaining=532, neg_remaining=561), proba=0.0059628480948123765, score=-0.21145791558213753, weighted_spans=None, heatmap=None), TargetExplanation(target='кримінал', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='вночі', weight=0.19920573138887007, std=None, value=0.2448583920018226), FeatureWeight(feature='область', weight=0.13384455926555497, std=None, value=0.12329526124930607), FeatureWeight(feature='решта', weight=0.08078285998258315, std=None, value=0.22182733515795455), FeatureWeight(feature='територія', weight=0.07626287625341284, std=None, value=0.1265692196449385), FeatureWeight(feature='узбережжя', weight=0.07357986822080034, std=None, value=0.14249321622206346)], neg=[FeatureWeight(feature='<BIAS>', weight=-0.3735951662965949, std=None, value=1.0), FeatureWeight(feature='градус', weight=-0.09142789005355904, std=None, value=0.2251043782160064), FeatureWeight(feature='температура', weight=-0.08938688607297325, std=None, value=0.2053642709121595), FeatureWeight(feature='більшість', weight=-0.08780540806097752, std=None, value=0.10321487284630691), FeatureWeight(feature='північний', weight=-0.07684326619460113, std=None, value=0.18439554835122274), FeatureWeight(feature='° ', weight=-0.061558181955640334, std=None, value=0.23236670124123585), FeatureWeight(feature='укргідрометцентру', weight=-0.059418032085263014, std=None, value=0.14769472157964184), FeatureWeight(feature='кінець', weight=-0.05651251555789185, std=None, value=0.09947279631444005), FeatureWeight(feature='південь', weight=-0.05363940437102902, std=None, value=0.13563461287011214), FeatureWeight(feature='градус тепла', weight=-0.05319385125271273, std=None, value=0.14014133876599444)], pos_remaining=604, neg_remaining=489), proba=0.005251139942189457, score=-0.3385609658943963, weighted_spans=None, heatmap=None), TargetExplanation(target='міжнародні новини', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=0.47068237157745657, std=None, value=1.0), Fe

In [24]:
# Saving the model
# sklearn models are not JSON serializable unfortunately
with open(os.path.join(PROJ_PATH, 'src', 'class_model_log.md'), 'wb') as f:
    pickle.dump(log_reg, f)